In [1]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [ ]:
# create list of all yyyy-mm for data download (Note, May 2015 is the first month ND data is available in xlxs)
myList = pd.date_range('2022-04-10',datetime.today()- relativedelta(months=11), freq='MS').strftime("%Y_%m").tolist()  # with month in numeric format

# Access the ND DMR data @ https://www.dmr.nd.gov/oilgas/mprindex.asp
website = "https://www.dmr.nd.gov/oilgas/mpr/"

In [ ]:
#myList

In [ ]:
NDMWD = []  #list()  # North Dakota Monthly Well Data
for month in myList:
    target_url = website + month + '.xlsx'
    print(target_url)
    r = requests.get(target_url)
    wb = pd.read_excel(r.content)
    NDMWD.append(wb)

NDMWD = pd.concat(NDMWD)

https://www.dmr.nd.gov/oilgas/mpr/2022_05.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_06.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_07.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_08.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_09.xlsx


In [ ]:
#Create ND monthly well data table in csv format, which can be downloaded from folder on left hand side
NDMWD.to_csv('NDMWD.csv', index=False)

In [27]:
NDMWD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72322 entries, 0 to 97297
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ReportDate  72322 non-null  datetime64[ns]
 1   API_WELLNO  72322 non-null  int64         
 2   FileNo      72322 non-null  int64         
 3   Company     72322 non-null  object        
 4   WellName    72322 non-null  object        
 5   Quarter     72296 non-null  object        
 6   Section     72322 non-null  int64         
 7   Township    72322 non-null  int64         
 8   Range       72322 non-null  int64         
 9   County      72322 non-null  object        
 10  FieldName   72322 non-null  object        
 11  Pool        72322 non-null  object        
 12  Oil         72322 non-null  float64       
 13  Wtr         72322 non-null  float64       
 14  Days        72322 non-null  float64       
 15  Runs        72322 non-null  int64         
 16  Gas         72322 non-

# **Extract first month oil and gas produciton**

In [72]:
#NDMWD = pd.read_csv('NDMWD.csv')
NDMWD.drop_duplicates(inplace=True)
NDMWD = NDMWD[NDMWD['Days'] >= 25].reset_index(drop=True)
NDMWD['ReportDate'] = pd.to_datetime(NDMWD['ReportDate'], errors = 'coerce')
NDMWD.tail(2)

,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,County,...,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
72311,2022-09-01,33009012640000,6347,EMPIRE NORTH DAKOTA LLC,RICE 2,SENW,26,163,79,BOT,...,SPEARFISH/MADISON,26.0,301.0,30.0,39,0.0,0,0.0,48.917191,-100.876462
72312,2022-09-01,33009019570000,13685,EMPIRE NORTH DAKOTA LLC,RICE 3,NENW,26,163,79,BOT,...,SPEARFISH/MADISON,46.0,5979.0,28.0,73,0.0,0,0.0,48.920808,-100.877109


In [73]:
ffmWells = NDMWD[['API_WELLNO', 'Pool','ReportDate']].groupby('API_WELLNO').min().rename(columns={'ReportDate':'ffMonth'}).reset_index()
ffmWells.tail(3)

,API_WELLNO,Pool,ffMonth
17551,33105056680000,BAKKEN,2022-05-01
17552,33105056760000,BAKKEN,2022-09-01
17553,33105056780000,BAKKEN,2022-09-01


In [74]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17554 entries, 0 to 17553
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  17554 non-null  int64         
 1   Pool        17554 non-null  object        
 2   ffMonth     17554 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 411.5+ KB


In [75]:
ffmWells = ffmWells.merge(NDMWD[['API_WELLNO','Pool','ReportDate','Oil','Gas','Wtr','Days','Runs', 'Lat', 'Long']], how='left', left_on=['API_WELLNO','Pool','ffMonth'], right_on = ['API_WELLNO','Pool','ReportDate'])
ffmWells.dropna(inplace=True)
ffmWells.reset_index(drop=True)
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,ReportDate,Oil,Gas,Wtr,Days,Runs,Lat,Long
17549,33105056660000,BAKKEN,2022-08-01,2022-08-01,17016.0,17776.0,41474.0,25.0,17052.0,48.460686,-103.525647
17550,33105056670000,BAKKEN,2022-09-01,2022-09-01,22311.0,21967.0,43614.0,26.0,21967.0,48.460686,-103.525750
17551,33105056680000,BAKKEN,2022-05-01,2022-05-01,15437.0,11578.0,37706.0,29.0,15196.0,48.531131,-103.473923
17552,33105056760000,BAKKEN,2022-09-01,2022-09-01,18888.0,15060.0,36179.0,29.0,19452.0,48.465588,-102.931861
17553,33105056780000,BAKKEN,2022-09-01,2022-09-01,18117.0,14845.0,30204.0,30.0,18246.0,48.465588,-102.931366


In [76]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17550 entries, 0 to 17553
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  17550 non-null  int64         
 1   Pool        17550 non-null  object        
 2   ffMonth     17550 non-null  datetime64[ns]
 3   ReportDate  17550 non-null  datetime64[ns]
 4   Oil         17550 non-null  float64       
 5   Gas         17550 non-null  float64       
 6   Wtr         17550 non-null  float64       
 7   Days        17550 non-null  float64       
 8   Runs        17550 non-null  float64       
 9   Lat         17550 non-null  float64       
 10  Long        17550 non-null  float64       
dtypes: datetime64[ns](2), float64(7), int64(1), object(1)
memory usage: 1.6+ MB


In [77]:
ffmWells.drop('ReportDate', axis=1, inplace=True )
ffmWells['bpd'] =  ffmWells.Oil/ffmWells.Days
ffmWells['Mcfpd'] =  ffmWells.Gas/ffmWells.Days

In [87]:
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,Oil,Gas,Wtr,Days,Runs,Lat,Long,bpd,Mcfpd,MHB
17549,33105056660000,BAKKEN,2022-08-01,17016.0,17776.0,41474.0,25.0,17052.0,48.460686,-103.525647,680.640000,711.040000,DN88fl
17550,33105056670000,BAKKEN,2022-09-01,22311.0,21967.0,43614.0,26.0,21967.0,48.460686,-103.525750,858.115385,844.884615,DN88fl
17551,33105056680000,BAKKEN,2022-05-01,15437.0,11578.0,37706.0,29.0,15196.0,48.531131,-103.473923,532.310345,399.241379,DN88gm
17552,33105056760000,BAKKEN,2022-09-01,18888.0,15060.0,36179.0,29.0,19452.0,48.465588,-102.931861,651.310345,519.310345,DN88ml
17553,33105056780000,BAKKEN,2022-09-01,18117.0,14845.0,30204.0,30.0,18246.0,48.465588,-102.931366,603.900000,494.833333,DN88ml


In [86]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17550 entries, 0 to 17553
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  17550 non-null  int64         
 1   Pool        17550 non-null  object        
 2   ffMonth     17550 non-null  datetime64[ns]
 3   Oil         17550 non-null  float64       
 4   Gas         17550 non-null  float64       
 5   Wtr         17550 non-null  float64       
 6   Days        17550 non-null  float64       
 7   Runs        17550 non-null  float64       
 8   Lat         17550 non-null  float64       
 9   Long        17550 non-null  float64       
 10  bpd         17550 non-null  float64       
 11  Mcfpd       17550 non-null  float64       
 12  MHB         17550 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 1.9+ MB


# **Wells to MHB allocation**

In [ ]:
# Maidenhead grid locators courtesy of Walter Underwood K6WRU:
# https://ham.stackexchange.com/questions/221/how-can-one-convert-from-lat-long-to-grid-square


In [81]:
upper = 'ABCDEFGHIJKLMNOPQRSTUVWX'
lower = 'abcdefghijklmnopqrstuvwx'

In [84]:
import sys

upper = 'ABCDEFGHIJKLMNOPQRSTUVWX'
lower = 'abcdefghijklmnopqrstuvwx'

def gridCalc(row):
    dec_lat = row['Lat']
    dec_lon = row['Long']
    if not (-180<=dec_lon<180):
        sys.stderr.write('longitude must be -180<=lon<180, given %f\n'%dec_lon)
        sys.exit(32)
    if not (-90<=dec_lat<90):
        sys.stderr.write('latitude must be -90<=lat<90, given %f\n'%dec_lat)
        sys.exit(33) # can't handle north pole, sorry, [A-R]

    adj_lat = dec_lat + 90.0
    adj_lon = dec_lon + 180.0

    grid_lat_sq = upper[int(adj_lat/10)];
    grid_lon_sq = upper[int(adj_lon/20)];

    grid_lat_field = str(int(adj_lat%10))
    grid_lon_field = str(int((adj_lon/2)%10))

    adj_lat_remainder = (adj_lat - int(adj_lat)) * 60
    adj_lon_remainder = ((adj_lon) - int(adj_lon/2)*2) * 60

    grid_lat_subsq = lower[int(adj_lat_remainder/2.5)]
    grid_lon_subsq = lower[int(adj_lon_remainder/5)]

    return grid_lon_sq + grid_lat_sq + grid_lon_field + grid_lat_field + grid_lon_subsq + grid_lat_subsq

In [85]:
ffmWells['MHB'] = ffmWells.apply(gridCalc, axis=1)

In [88]:
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,Oil,Gas,Wtr,Days,Runs,Lat,Long,bpd,Mcfpd,MHB
17549,33105056660000,BAKKEN,2022-08-01,17016.0,17776.0,41474.0,25.0,17052.0,48.460686,-103.525647,680.640000,711.040000,DN88fl
17550,33105056670000,BAKKEN,2022-09-01,22311.0,21967.0,43614.0,26.0,21967.0,48.460686,-103.525750,858.115385,844.884615,DN88fl
17551,33105056680000,BAKKEN,2022-05-01,15437.0,11578.0,37706.0,29.0,15196.0,48.531131,-103.473923,532.310345,399.241379,DN88gm
17552,33105056760000,BAKKEN,2022-09-01,18888.0,15060.0,36179.0,29.0,19452.0,48.465588,-102.931861,651.310345,519.310345,DN88ml
17553,33105056780000,BAKKEN,2022-09-01,18117.0,14845.0,30204.0,30.0,18246.0,48.465588,-102.931366,603.900000,494.833333,DN88ml


In [89]:
ffmWells.to_csv('ffmWells.csv', index=False)